In [1]:
import mlrose_hiive as mlh
import numpy as np
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
import random

In [2]:
# Load the data
dataset = pd.read_csv('ks-projects-201801.csv')
le = preprocessing.LabelEncoder()
ohe =preprocessing.OneHotEncoder()

dataset['launched']=pd.to_datetime(dataset.launched)
dataset['deadline']=pd.to_datetime(dataset.deadline)
dataset['start_to_deadline']=(dataset.deadline-dataset.launched).astype('timedelta64[h]') 

#Preprocess
dataset['category']=le.fit_transform(dataset.category.values)
dataset['main_category']=le.fit_transform(dataset.main_category.values)
dataset['currency']=le.fit_transform(dataset.currency.values)
dataset['country']=le.fit_transform(dataset.country.values)



columns_to_keep=['goal','pledged','backers','usd_pledged_real','usd_goal_real','start_to_deadline']
states_to_keep=['failed','successful']
dataset_limit_columns=dataset[columns_to_keep+['state']]
dataset_limit_outcomes=dataset[dataset['state'].isin(states_to_keep)]


dataset_sample=dataset_limit_outcomes.iloc[random.sample(range(0,len(dataset_limit_outcomes)),10000)]

# Train test split
train,test=train_test_split(dataset_sample, test_size=0.2, random_state=40)
train_x,train_y=train[columns_to_keep],train['state']
test_x,test_y=test[columns_to_keep],test['state']

# Normalize feature data
scaler = preprocessing.MinMaxScaler()

train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)

# train_y_hot = ohe.fit_transform(np.asarray(train_y).reshape(-1, 1)).todense()
# test_y_hot = ohe.transform(np.asarray(test_y).reshape(-1, 1)).todense()

train_y_hot=le.fit_transform(train_y)
test_y_hot=le.fit_transform(test_y)



In [3]:
# Network params
max_iters=[100,200]#,500]
learning_rates =[1e-3,1e-4]#,1e-5]#,1e-5]#,0.1]
clip_maxs=[1,5]#,100]#1e1,1e2,1e3,1e5,1e10]

def fit_model_verbose(model,i,n):
    print('Iteration ' + str(i) + ' of ' + str(n-1))
    start=time.time()
    model.fit(train_x_scaled,train_y_hot)
    end=time.time()
    print(pd.DataFrame(model.predict(train_x_scaled))[0].value_counts())
    return(end-start)
    
def get_results(model,train_x,train_y,test_x,test_y):
    predictions=model.predict(train_x)
    train_accuracy=accuracy_score(predictions,train_y)
    test_accuracy=accuracy_score(model.predict(test_x),test_y)
    perc_class1=pd.DataFrame(predictions)[0].value_counts()[0]/len(train_x)
    results=pd.DataFrame(data={'train_accuracy':[train_accuracy],'test_accuracy':[test_accuracy],'loss':[model.loss],'fitted_weights':[model.fitted_weights],'fitness_curve':[model.fitness_curve],'perc_class1':[perc_class1]})
    return(results)

In [31]:

# Random Hill Climb
restarts =[1,2,10,100]
max_attempts=[10,25,50,100,250]

params_rhc=pd.concat([pd.DataFrame(data={'clip_max':[clip_max],'restarts':[restart],'max_attempts':[max_attempt],'learning_rate':[learning_rate],'max_iters':[max_iter]}) for clip_max in clip_maxs for restart in restarts for max_attempt in max_attempts for learning_rate in learning_rates for max_iter in max_iters]) 

models_rhc=[mlh.NeuralNetwork(hidden_nodes = [5,5,5,5,5], activation = 'relu', \
                                 algorithm = 'random_hill_climb', max_iters = params_rhc.iloc[i]['max_iters'], \
                                 bias = True, is_classifier = True, learning_rate =  params_rhc.iloc[i]['learning_rate'], \
                                 restarts=int(params_rhc.iloc[i]['restarts']),early_stopping = True, clip_max =  params_rhc.iloc[i]['clip_max'], max_attempts =  params_rhc.iloc[i]['max_attempts'], \
                                 random_state = 3,curve=True) for i in range(len(params_rhc))]

fit_times=[fit_model_verbose(models_rhc[i],i,len(models_rhc)) for i in range(len(models_rhc))]

results=pd.concat([get_results(model,train_x_scaled,train_y_hot,test_x_scaled,test_y_hot) for model in models_rhc])
results=pd.concat([params_rhc,results],axis=1)
results['fit_time']=fit_times
results.to_csv('results_nn/output_rhc_pd.csv')



Iteration 0 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 1 of 359
1    7992
0       8
Name: 0, dtype: int64
Iteration 2 of 359
1    7986
0      14
Name: 0, dtype: int64
Iteration 3 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 4 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 5 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 6 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 7 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 8 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 9 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 10 of 359
1    7992
0       8
Name: 0, dtype: int64
Iteration 11 of 359
1    7986
0      14
Name: 0, dtype: int64
Iteration 12 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 13 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 14 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 15 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 16 of 35

1    7995
0       5
Name: 0, dtype: int64
Iteration 133 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 134 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 135 of 359
1    7993
0       7
Name: 0, dtype: int64
Iteration 136 of 359
1    7992
0       8
Name: 0, dtype: int64
Iteration 137 of 359
1    7986
0      14
Name: 0, dtype: int64
Iteration 138 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 139 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 140 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 141 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 142 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 143 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 144 of 359
1    7993
0       7
Name: 0, dtype: int64
Iteration 145 of 359
1    7992
0       8
Name: 0, dtype: int64
Iteration 146 of 359
1    7986
0      14
Name: 0, dtype: int64
Iteration 147 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 148

1    7986
0      14
Name: 0, dtype: int64
Iteration 264 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 265 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 266 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 267 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 268 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 269 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 270 of 359
1    7993
0       7
Name: 0, dtype: int64
Iteration 271 of 359
1    7992
0       8
Name: 0, dtype: int64
Iteration 272 of 359
1    7986
0      14
Name: 0, dtype: int64
Iteration 273 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 274 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 275 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 276 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 277 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 278 of 359
1    7995
0       5
Name: 0, dtype: int64
Iteration 279

In [33]:
# Simulated Annealing
init_temps=[3,5,10]#,20]
decays=[0.6,0.8,0.9]#,0.95]#,0.99]
schedules=[fnc(init_temp=init_temp, decay=decay, min_temp=0) for init_temp in init_temps for decay in decays for fnc in[mlh.GeomDecay]]
max_attempts=[500,1000,2000]

params_sa=pd.concat([pd.DataFrame(data={'clip_max':[clip_max],'init_temp':[init_temp],'decay':[decay],'max_attempts':[max_attempt],'learning_rate':[learning_rate],'max_iters':[max_iter]}) for clip_max in clip_maxs for init_temp in init_temps for decay in decays for max_attempt in max_attempts for learning_rate in learning_rates for max_iter in max_iters]) 

models_sa=[mlh.NeuralNetwork(hidden_nodes = [5,5,5,5,5], activation = 'relu', \
                                 algorithm = 'simulated_annealing', max_iters = params_sa.iloc[i]['max_iters'], \
                                 bias = True, is_classifier = True, learning_rate =  params_sa.iloc[i]['learning_rate'], \
                                 early_stopping = True, clip_max =  params_sa.iloc[i]['clip_max'], max_attempts =  params_sa.iloc[i]['max_attempts'], \
                                 random_state = 3,curve=True) for i in range(len(params_sa))]

fit_times=[fit_model_verbose(models_sa[i],i,len(models_sa)) for i in range(len(models_sa))]

results=pd.concat([get_results(model,train_x_scaled,train_y_hot,test_x_scaled,test_y_hot) for model in models_sa])
results=pd.concat([params_sa,results],axis=1)
results['fit_time']=fit_times
results.to_csv('results_nn/output_sa_pd.csv')


In [5]:
# genetic
max_attempts=[10,50,100]
pop_sizes=[100,250]
mutation_probs=[0.1,0.2,0.3]#,0.4]

# params_ga=pd.concat([pd.DataFrame(data={'clip_max':[clip_max],'pop_size':[pop_size],'mutation_prob':[mutation_prob],'max_attempts':[max_attempt],'learning_rate':[learning_rate],'max_iters':[max_iter]}) for clip_max in clip_maxs for pop_size in pop_sizes for mutation_prob in mutation_probs for max_attempt in max_attempts for learning_rate in learning_rates for max_iter in max_iters]) 

# models_ga=[mlh.NeuralNetwork(hidden_nodes = [5,5,5,5,5], activation = 'relu', \
#                                  algorithm = 'genetic_alg', max_iters = params_ga.iloc[i]['max_iters'], \
#                                  bias = True, is_classifier = True, learning_rate =  params_ga.iloc[i]['learning_rate'], \
#                                  early_stopping = True, clip_max =  params_ga.iloc[i]['clip_max'], max_attempts =  params_ga.iloc[i]['max_attempts'], \
#                                  pop_size= params_ga.iloc[i]['pop_size'],mutation_prob= params_ga.iloc[i]['mutation_prob'],random_state = 3,curve=True) for i in range(len(params_ga))]

# fit_times=[fit_model_verbose(models_ga[i],i,len(models_ga)) for i in range(len(models_ga))]

results=pd.concat([get_results(model,train_x_scaled,train_y_hot,test_x_scaled,test_y_hot) for model in models_ga])
results=pd.concat([params_ga,results],axis=1)
results['fit_time']=fit_times
results.to_csv('results_nn/output_ga_pd.csv')




In [30]:
# gradient descent
params_gd=pd.concat([pd.DataFrame(data={'clip_max':[clip_max],'learning_rate':[learning_rate],'max_iters':[max_iter]}) for clip_max in clip_maxs  for learning_rate in learning_rates for max_iter in max_iters]) 

models_gd=[mlh.NeuralNetwork(hidden_nodes =tuple([5]*5), activation = 'relu', \
                                 algorithm = 'gradient_descent', max_iters = params_gd.iloc[i]['max_iters'], \
                                 bias = True, is_classifier = True, learning_rate =  params_gd.iloc[i]['learning_rate'], \
                                 early_stopping = True, clip_max =  params_gd.iloc[i]['clip_max'], \
                                 random_state = 3,curve=True) for i in range(len(params_gd))]

fit_times=[fit_model_verbose(models_gd[i],i,len(models_gd)) for i in range(len(models_gd))]

results=pd.concat([get_results(model,train_x_scaled,train_y_hot,test_x_scaled,test_y_hot) for model in models_gd])
results=pd.concat([params_gd,results],axis=1)
results['fit_time']=fit_times
results.to_csv('results_nn/output_gd_pd.csv')

results

Iteration 0 of 17
0    7944
1      56
Name: 0, dtype: int64
Iteration 1 of 17
0    7998
1       2
Name: 0, dtype: int64
Iteration 2 of 17
0    7998
1       2
Name: 0, dtype: int64
Iteration 3 of 17
0    7745
1     255
Name: 0, dtype: int64
Iteration 4 of 17
0    7735
1     265
Name: 0, dtype: int64
Iteration 5 of 17
0    7693
1     307
Name: 0, dtype: int64
Iteration 6 of 17
0    7095
1     905
Name: 0, dtype: int64
Iteration 7 of 17
0    7641
1     359
Name: 0, dtype: int64
Iteration 8 of 17
0    7642
1     358
Name: 0, dtype: int64
Iteration 9 of 17
0    7945
1      55
Name: 0, dtype: int64
Iteration 10 of 17
0    7999
1       1
Name: 0, dtype: int64
Iteration 11 of 17
0    7999
1       1
Name: 0, dtype: int64
Iteration 12 of 17
0    7745
1     255
Name: 0, dtype: int64
Iteration 13 of 17
0    7735
1     265
Name: 0, dtype: int64
Iteration 14 of 17
0    7693
1     307
Name: 0, dtype: int64
Iteration 15 of 17
0    7095
1     905
Name: 0, dtype: int64
Iteration 16 of 17
0    7641
1    

clip_max  learning_rate  max_iters  train_accuracy  test_accuracy  \
0         1        0.00100         50        0.608375         0.5840   
0         1        0.00100        100        0.610375         0.5865   
0         1        0.00100        200        0.610375         0.5865   
0         1        0.00010         50        0.612250         0.5885   
0         1        0.00010        100        0.613500         0.5885   
0         1        0.00010        200        0.615000         0.5925   
0         1        0.00001         50        0.559000         0.5340   
0         1        0.00001        100        0.608750         0.5830   
0         1        0.00001        200        0.608875         0.5830   
0         5        0.00100         50        0.608250         0.5840   
0         5        0.00100        100        0.610250         0.5865   
0         5        0.00100        200        0.610250         0.5865   
0         5        0.00010         50        0.612250         0.5885   
0         5        0.00010        100        0.613500         0.5885   
0         5        0.00010        200        0.615000         0.5925   
0         5        0.00001         50        0.559000         0.5340   
0         5        0.00001        100        0.608750         0.5830   
0         5        0.00001        200        0.608875         0.5830   

       loss                                     fitted_weights  \
0  0.693147  [0.10106191244134592, 0.41645418911011317, -0....   
0  0.693145  [0.10105271241583226, 0.4164535654237809, -0.4...   
0  0.693094  [0.10138458869359977, 0.41505848815242796, -0....   
0  0.693089  [0.10084487245910911, 0.4168126963655866, -0.4...   
0  0.693047  [0.10010511021191984, 0.4173201662728794, -0.4...   
0  0.692911  [0.09839360451493037, 0.41844737471707083, -0....   
0  0.693171  [0.10145276193779217, 0.4163649402186808, -0.4...   
0  0.693110  [0.10135400540201435, 0.4164237455719308, -0.4...   
0  0.693097  [0.10116041713750924, 0.41653841916773554, -0....   
0  0.693147  [0.10106655589332758, 0.4164541685267266, -0.4...   
0  0.693145  [0.10106582361716095, 0.4164534673626015, -0.4...   
0  0.693087  [0.10147706875963507, 0.41475420003277413, -0....   
0  0.693089  [0.10084487245910911, 0.4168126963655866, -0.4...   
0  0.693047  [0.10010511021191984, 0.4173201662728794, -0.4...   
0  0.692911  [0.09839360451493037, 0.41844737471707083, -0....   
0  0.693171  [0.10145276193779217, 0.4163649402186808, -0.4...   
0  0.693110  [0.10135400540201435, 0.4164237455719308, -0.4...   
0  0.693097  [0.10116041713750924, 0.41653841916773554, -0....   

                                       fitness_curve  perc_class1  fit_time  
0  [-0.7047458203322085, -0.6931485724674955, -0....     0.993000  0.892466  
0  [-0.7047458203322085, -0.6931485724674955, -0....     0.999750  1.545032  
0  [-0.7047458203322085, -0.6931485724674955, -0....     0.999750  2.715269  
0  [-0.7047458203322085, -0.6943918805323686, -0....     0.968125  0.708917  
0  [-0.7047458203322085, -0.6943918805323686, -0....     0.966875  1.391047  
0  [-0.7047458203322085, -0.6943918805323686, -0....     0.961625  2.640034  
0  [-0.7047458203322085, -0.7028120768950827, -0....     0.886875  0.717718  
0  [-0.7047458203322085, -0.7028120768950827, -0....     0.955125  1.327873  
0  [-0.7047458203322085, -0.7028120768950827, -0....     0.955250  2.746159  
0  [-0.7047458203322085, -0.6931488678269259, -0....     0.993125  0.676347  
0  [-0.7047458203322085, -0.6931488678269259, -0....     0.999875  1.310947  
0  [-0.7047458203322085, -0.6931488678269259, -0....     0.999875  2.474401  
0  [-0.7047458203322085, -0.6943918805323686, -0....     0.968125  0.667436  
0  [-0.7047458203322085, -0.6943918805323686, -0....     0.966875  1.286508  
0  [-0.7047458203322085, -0.6943918805323686, -0....     0.961625  2.693283  
0  [-0.7047458203322085, -0.7028120768950827, -0....     0.886875  0.670660  
0  [-0.7047458203322085, -0.7028120768950827, -0.... 

In [6]:
results

clip_max  pop_size  mutation_prob  max_attempts  learning_rate  max_iters  \
0          1       100            0.1            10         0.0010        100   
0          1       100            0.1            10         0.0010        200   
0          1       100            0.1            10         0.0001        100   
0          1       100            0.1            10         0.0001        200   
0          1       100            0.1            50         0.0010        100   
..       ...       ...            ...           ...            ...        ...   
0          5       250            0.3            50         0.0001        200   
0          5       250            0.3           100         0.0010        100   
0          5       250            0.3           100         0.0010        200   
0          5       250            0.3           100         0.0001        100   
0          5       250            0.3           100         0.0001        200   

    train_accuracy  test_accuracy      loss  \
0          0.59050         0.6025  0.660448   
0          0.59050         0.6025  0.660448   
0          0.59050         0.6025  0.660448   
0          0.59050         0.6025  0.660448   
0          0.59050         0.6025  0.660448   
..             ...            ...       ...   
0          0.59100         0.6025  0.667293   
0          0.59100         0.6025  0.667293   
0          0.62075         0.6295  0.662405   
0          0.59100         0.6025  0.667293   
0          0.62075         0.6295  0.662405   

                                       fitted_weights  \
0   [0.6900660027803425, -0.8451866430088866, -0.1...   
0   [0.6900660027803425, -0.8451866430088866, -0.1...   
0   [0.6900660027803425, -0.8451866430088866, -0.1...   
0   [0.6900660027803425, -0.8451866430088866, -0.1...   
0   [0.6900660027803425, -0.8451866430088866, -0.1...   
..                                                ...   
0   [0.3256740245417289, 2.8637094035681354, -3.76...   
0   [0.3256740245417289, 2.8637094035681354, -3.76...   
0   [-4.381410547291196, -1.4071586686175097, 4.80...   
0   [0.3256740245417289, 2.8637094035681354, -3.76...   
0   [-4.381410547291196, -1.4071586686175097, 4.80...   

                                        fitness_curve  perc_class1    fit_time  
0   [0.6732587133983499, 0.6732587133983499, 0.672...      1.00000   15.114763  
0   [0.6732587133983499, 0.6732587133983499, 0.672...      1.00000   13.984543  
0   [0.6732587133983499, 0.6732587133983499, 0.672...      1.00000   14.168580  
0   [0.6732587133983499, 0.6732587133983499, 0.672...      1.00000   14.035483  
0   [0.6732587133983499, 0.6732587133983499, 0.672...      1.00000   32.589124  
..                                                ...          ...         ...  
0   [0.6912410482326389, 0.6912410482326389, 0.691...      0.99950  111.069597  
0   [0.6912410482326389, 0.6912410482326389, 0.691...      0.99950  119.867882  
0   [0.6912410482326389, 0.6912410482326389, 0.691...      0.96925  237.343118  
0   [0.6912410482326389, 0.6912410482326389, 0.691...      0.99950  119.765364  
0   [0.6912410482326389, 0.6912410482326389, 0.691...      0.96925  238.024239  

[144 rows x 13 columns]